In [2]:
import sys
# first time running you may need to uncomment the install line below
# !{sys.executable} -m pip install yfinance
import pandas as pd

#!{sys.executable} -m pip install pandas_datareader
#import pandas_datareader as pdr
import yfinance as yf

# change `AX` to whatever stock you want more information on
lookup = "AX"


In [18]:
tick = yf.Ticker(lookup)

e = tick.earnings
yoy_metrics = {}
# print(tick.earnings["Revenue"], tick.earnings["Earnings"])


for date, earnings, in e["Earnings"].items():
    yoy_metrics[str(date)] = {"Earnings": earnings}

for date, rev in e["Revenue"].items():
    yoy_metrics[str(date)]["Revenue"] = rev

for title, data in tick.cash_flow.iterrows():
    if title == "Operating Cash Flow":
        
        for date, amt in data.items():
            yoy_metrics[str(date.year)]["Cash Flow"] = amt
        

# Display the shizzing

print(f"Stock: {lookup} (Amounts in thousands)")
print("\tEarnings\tRevenue\t\tOperating Cash Flow")

for y in yoy_metrics.keys():
    pme = f"""{y}\t${yoy_metrics[y]["Earnings"]//1000:,} T\t${yoy_metrics[y]["Revenue"]//1000:,} T\t${yoy_metrics[y]["Cash Flow"]//1000:,} T"""
    print(pme)

    
guidelines = """GUIDELINES FOR INVESTMENT CLUB:
MARKET CAP WITHIN 1 - 10 B RANGE, and 
15 % GROWTH EACH YEAR IN: 
    - EARNINGS
    - REVENUE
    - OPERATING CASH FLOW
"""
print(guidelines)

calculations = {}
for y in list(yoy_metrics.keys())[1:]:
    prior = str(int(y) - 1)
    diff_key = f"{prior} - {y}"
    earn_diff = (yoy_metrics[y]["Earnings"] - yoy_metrics[prior]["Earnings"]) / yoy_metrics[prior]["Earnings"] * 100
    rev_diff = (yoy_metrics[y]["Revenue"] - yoy_metrics[prior]["Revenue"]) / yoy_metrics[prior]["Revenue"] * 100
    opcash_diff = (yoy_metrics[y]["Cash Flow"] - yoy_metrics[prior]["Cash Flow"]) / yoy_metrics[prior]["Cash Flow"] * 100
    calculations[diff_key] = {"Earnings":earn_diff, "Revenue":rev_diff, "Cash Flow":opcash_diff}

print("Year\t\tEarnings\tRevenue\t\tOperating Cash Flow")

for y in calculations.keys():
    pme = f"""{y}\t{calculations[y]['Earnings']:.2f}%\t\t{calculations[y]["Revenue"]:.2f}%\t\t{calculations[y]["Cash Flow"]:.2f}%"""
    print(pme)

Stock: AX (Amounts in thousands)
	Earnings	Revenue		Operating Cash Flow
2019	$155,131 T	$464,012 T	$204,421.0 T
2020	$183,438 T	$538,398 T	$284,118.0 T
2021	$215,707 T	$620,253 T	$412,582.0 T
2022	$240,716 T	$702,021 T	$210,282.0 T
GUIDELINES FOR INVESTMENT CLUB:
MARKET CAP WITHIN 1 - 10 B RANGE, and 
15 % GROWTH EACH YEAR IN: 
    - EARNINGS
    - REVENUE
    - OPERATING CASH FLOW

Year		Earnings	Revenue		Operating Cash Flow
2019 - 2020	18.25%		16.03%		38.99%
2020 - 2021	17.59%		15.20%		45.22%
2021 - 2022	11.59%		13.18%		-49.03%
